In [ ]:
import os
import shutil
import uuid
import xml.etree.ElementTree as ET

import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
from PIL import ImageOps

# Пути для сохранения разделенных выборок
images_train_dir = "../../data/processed/2 For OCR/train"
images_test_dir = "../../data/processed/2 For OCR/test"


# Создание каталогов для train, test
os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_test_dir, exist_ok=True)

# Список для хранения путей к изображениям и разметкам
images = []
annotations = []

###############################################################################
############# Формируем датасет из каталога Губернаторские отчеты #############
###############################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Губернаторские отчеты"

# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".JPG"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".JPG", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)

########################################################################################
############# Формируем датасет из каталога Уставные грамоты – Афанасенков #############
########################################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Уставные грамоты – Афанасенков"
# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".jpg", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)

############################################################################################
############# Формируем датасет из каталога Уставные грамоты в jpg (Просветов) #############
############################################################################################

image_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)"
annotation_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)/Обработка/Просветов (13.12)"

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_file = file.replace(".jpg", "_pvoc_imglab.xml")
            annotation_path = os.path.join(annotation_dir, annotation_file)
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)


# Разделение выборки на train, valid и test
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.1, random_state=42)

# Функция для копирования файлов
def copy_files(images, annotations, dest_dir):
    for image, annotation in zip(images, annotations):
        shutil.copy(image, dest_dir)
        shutil.copy(annotation, dest_dir)

# Функция для преобразования координат в относительные координаты
def convert_coordinates(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x_rel = x * dw
    w_rel = w * dw
    y_rel = y * dh
    h_rel = h * dh
    return x_rel, y_rel, w_rel, h_rel


# Преобразование разметки для train выборки
def process_data(images, annotations, image_save_path, data_type):
    
    # Счётчик количества ошибок в разметке боксов
    err_count = 0
    
    # Датафрейм для датасета
    data = pd.DataFrame()
    
    for image_path, annotation_path in tqdm(zip(images, annotations), total=len(images), desc=f"Подготавливаем {data_type} выборку"):
        
        # Преобразование разметки в формат YOLO
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        objects = []
        for obj in root.findall("object"):
            segment_name = str(uuid.uuid4()) + ".JPG"
            name = obj.find("name").text
            bbox = [
                float(obj.find("bndbox/xmin").text),
                float(obj.find("bndbox/ymin").text),
                float(obj.find("bndbox/xmax").text),
                float(obj.find("bndbox/ymax").text),
            ]
            
            img = Image.open(image_path)
            img = ImageOps.exif_transpose(img)
            
            try:
                cropped_segment = img.crop(bbox)
                cropped_segment.save(os.path.join(image_save_path, segment_name))
                
                objects.append({"file_name": segment_name, "text": name})
                
            # могут быть ошибки разметки и DecompressionBombError
            except Exception:
                err_count += 1
                continue
        
        # Добавляем полученные данные в датафрейм 
        new_data = pd.DataFrame(data=objects)
        data = pd.concat([data, new_data])
    
    print(f"Количество ошибок в разметке боксов: {err_count}")
    
    return data
            
# Обработка данных для каждой выборки
train_dataframe = process_data(train_images, train_annotations, images_train_dir, "обучающую")
test_dataframe = process_data(test_images, test_annotations, images_test_dir, "тестовую")

In [ ]:
train_dataframe.to_csv("../../data/processed/2 For OCR/train.csv")
test_dataframe.to_csv("../../data/processed/2 For OCR/test.csv")

In [ ]:
f"Размер обучающей выборки: {len(train_dataframe)} | Размер тестовой выборки: {len(test_dataframe)}"